NPYファイルを読み込みRVMで線形補間<br>
PTHファイルを読み込みNPYに

In [9]:
# 必要なライブラリをインポートする
import numpy as np
import os
import torch
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from tqdm import notebook
import joblib

#DINOの出力PATH
DATASET_PATH = "DINO_OUTPUT/0328cenHEAD_epo15"
file_type = "train"

feature_data_pth = torch.load(os.path.join(DATASET_PATH, file_type+"feat.pth"))
label_data_pth = torch.load(os.path.join(DATASET_PATH, file_type+"labels.pth"))
np.save(os.path.join(DATASET_PATH, file_type+"feat.npy"), feature_data_pth)
np.save(os.path.join(DATASET_PATH, file_type+"labels.npy"), label_data_pth)

feature_data_npy = np.load(os.path.join(DATASET_PATH, file_type+"feat.npy"))
label_data_npy = np.load(os.path.join(DATASET_PATH, file_type+"labels.npy"))

# データを読み込む
feature_data_npy = np.load(os.path.join(DATASET_PATH, file_type+"feat.npy"))
label_data_npy = np.load(os.path.join(DATASET_PATH, file_type+"labels.npy"))

# データを正規化する（重要な前処理手順）
scaler = StandardScaler()
feature_data_npy = scaler.fit_transform(feature_data_npy)

# 訓練データとテストデータに分割する
X_train, X_test, y_train, y_test = train_test_split(feature_data_npy, label_data_npy, test_size=0.2, random_state=42)

print("feature shape", feature_data_npy.shape)
print("Train shape:", X_train.shape, "Valid shape", X_test.shape)


feature shape (12000, 768)
Train shape: (9600, 768) Test shape (2400, 768)


SVR学習

In [2]:
# SVRモデルを作成する
svr_model = SVR(kernel='rbf', C=1.0, epsilon=0.1)

# プログレスバーを表示しながらモデルを訓練する
num_epochs = 100
for epoch in notebook.tqdm(range(num_epochs), desc=file_type+"ing"):
    svr_model.fit(X_train, y_train)

# テストデータでモデルを評価する
predictions = svr_model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
print("Mean Squared Error:", mse)

#model 保存
save_path = os.path.join(DATASET_PATH, "SVR")
if os.path.exists(save_path)==False:
    os.mkdir(save_path)
joblib.dump(svr_model, os.path.join(save_path, file_type+'svr_model.pkl'))

training:   0%|          | 0/100 [00:00<?, ?it/s]

Mean Squared Error: 0.5878405778546286


['DINO_OUTPUT/0328cenHEAD_epo15/SVR/trainsvr_model.pkl']

学習後にモデル検証

In [8]:
# モデルの読み込み
file_type = "train"
save_path = os.path.join(DATASET_PATH, "SVR")
file_path = os.path.join(save_path, file_type+'svr_model.pkl')
model = joblib.load(file_path)

# データを読み込む
file_type = "test"
eature_data_pth = torch.load(os.path.join(DATASET_PATH, file_type+"feat.pth"))
label_data_pth = torch.load(os.path.join(DATASET_PATH, file_type+"labels.pth"))
np.save(os.path.join(DATASET_PATH, file_type+"feat.npy"), feature_data_pth)
np.save(os.path.join(DATASET_PATH, file_type+"labels.npy"), label_data_pth)
feature_data_npy = np.load(os.path.join(DATASET_PATH, file_type+"feat.npy"))
label_data_npy = np.load(os.path.join(DATASET_PATH, file_type+"labels.npy"))
print("TEST data shape", feature_data_npy.shape)
print("label_data_npy", label_data_npy.shape)

# データを正規化する（重要な前処理手順）
scaler = StandardScaler()
feature_data_npy = scaler.fit_transform(feature_data_npy)

#推論
from tqdm import tqdm
predictions = []
for i in tqdm(range(len(feature_data_npy)), desc="Predicting"):
    prediction = model.predict([feature_data_npy[i]])
    predictions.append(prediction)
    

print("prediction shape", len(predictions))

mse = mean_squared_error(label_data_npy, predictions)

print(predictions)
print(y_test)
print("Mean Squared Error:", mse)


TEST data shape (12000, 768)
label_data_npy (14320,)


Predicting:   9%|▉         | 1127/12000 [00:03<00:29, 363.49it/s]


KeyboardInterrupt: 